In [52]:
llm_file = "../../arena/evaluation_sanitized-mbpp.json"

In [53]:
import json

file = open(llm_file, 'r')
tasks = json.load(file)

In [54]:
task = tasks[0]
tests = task["test_list"]
test = tests[0]

In [55]:
import ast

test_ast = ast.parse(test).body[0]

In [30]:
ast.get_source_segment(test, test_ast.test.left)

'set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10)))'

In [45]:
import builtins

def create_sequence_sheet_entries(test, element):
    global cell_number
    sequence_sheet = []
    if type(element) == ast.Call:
        args = []
        for arg in element.args:
            if type(arg) == ast.Call:
                sequence_sheet += create_sequence_sheet_entries(test, arg)
                args.append(f"A{cell_number-1}")
            else:
                args.append(ast.get_source_segment(test, arg))
        function_name = element.func.id
        if function_name in dir(builtins):
            function_name = "python." + function_name
        sequence_sheet.append([f"A{cell_number}", function_name] + [args])
    cell_number += 1
    return sequence_sheet

cell_number = 1
create_sequence_sheet_entries(test, test_ast.test.left)

[['A1', 'similar_elements', ['(3, 4, 5, 6)', '(5, 7, 4, 10)']],
 ['A2', 'python.set', ['A1']]]

In [46]:
cell_number = 1
create_sequence_sheet_entries(test, test_ast.test.comparators[0])

[['A1', 'python.set', ['(4, 5)']]]

In [60]:
sequence_sheets = []
for task in tasks[:2]:
    tests = task["test_list"]
    sequence_sheet = []
    for test in tests:
        test_ast = ast.parse(test).body[0].test
        if type(test_ast) == ast.Compare:
            cell_number = 1
            left = create_sequence_sheet_entries(test, test_ast.left)
            cell_number = 1
            right = create_sequence_sheet_entries(test, test_ast.comparators[0])
            sequence_sheet += (left + right)
    sequence_sheets.append(sequence_sheet)